<a href="https://colab.research.google.com/github/arutraj/ML_Basics/blob/main/15_4_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning

<tr>
    <td><img src="images/imagenet_challenge.png" alt="drawing" style="width:500px;" align="left"/></td>
    <td><img src="images/screenshot.png" alt="drawing" style="width:500px;" align="left"/></td>
</tr>

## Classification Problem - Emergency vs Non-emergency Vehicle Classification

<img src='images/emergency_vs_non.png'>


[Source](https://www.slideshare.net/xavigiro/image-classification-on-imagenet-d1l4-2017-upc-deep-learning-for-computer-vision/)

Table of Contents
1. <font color='blue'>Import neccessary libraries
2. Load the data</font>
3. <font color='#999900'>Preprocess the data</font>
4. <font color='red'>Load weights of pre-trained model
5. Fine tune the model for the current problem</font>
6. <font color='blue'>Get predictions</font>

## 1. Import neccessary libraries

Here we will import the required libraries and functions.

In [2]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from skimage.transform import resize

from tensorflow.keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import ResNet50
from keras.models import Sequential
from keras.layers import Dense, InputLayer

## 2. Load the data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
#!unzip /content/drive/MyDrive/AV/Images/Dataset.zip
!ls /content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/*jpg |wc -l

2352


In [7]:
#reading the csv file containing data labels
data = pd.read_csv('/content/emergency_classification.csv')

In [8]:
data.head()

,image_names,emergency_or_not
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [9]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('/content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/' + img_name)
    # pile it one over the other
    X.append(img)

# convert this python list to a single numpy array
X = np.array(X)

In [10]:
#getting the labels for images
y = data['emergency_or_not'].values

In [15]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [11]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency ,
#with only one of them is true for every image
y = to_categorical(y)

In [14]:
y

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

## 3. Pre-Process Data

Steps :
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [28]:
#show maximum and minimum values for the image array
X.min(), X.max()

(-233.558, 27.381004)

In [27]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X, data_format=None)
#X = preprocess_input(X, mode='tf')
#preprocess_input?

In [29]:
X.min(), X.max()

(-233.558, 27.381004)

In [30]:
#print minimum and maximum values present in the array
X.min(), X.max()

(-233.558, 27.381004)

In [31]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load weights of pretrained model

In [32]:
# creating model with pre trained imagenet weights
base_model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 24s 0us/step


In [33]:
#shows model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

<img src="images/original_vgg_model_input.png" alt="drawing" style="width:500px;"/>

<img src="images/original_vgg_model_knowledge_layer.png" alt="drawing" style="width:500px;"/>

<img src="images/original_vgg_model_top_layer.png" alt="drawing" style="width:500px;"/>

In [34]:
# creating a VGG16 model with imagenet pretrained weights , accepting input of shape (224,224,3)
# also remove the final layers from model(include_top= False)
base_model = VGG16(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

58889256/58889256 [==============================] - 4s 0us/step


In [35]:
# show model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

## 5. Fine tune the model for the current problem
Steps:-
1. Extract features
2. Flatten the data
3. Rescale features
4. Create a Neural Network Model
5. Compile the model
6. Train and Validate the model

In [36]:
# extract features using the pretrained VGG16 model
# for training set
base_model_pred = base_model.predict(X_train)
#for validation set
base_model_pred_valid = base_model.predict(X_valid)

23/23 [==============================] - 4s 173ms/step


In [37]:
#show shape of predictions
base_model_pred.shape

(1646, 7, 7, 512)

In [38]:
# flattening the model output to one dimension for every sample of training set
base_model_pred = base_model_pred.reshape(1646, 7*7*512)

In [39]:
base_model_pred.shape

(1646, 25088)

In [40]:
base_model_pred_valid.shape

(706, 7, 7, 512)

In [41]:
# flattening the model output to one dimension for every sample of validation set
base_model_pred_valid = base_model_pred_valid.reshape(706, 7*7*512)

In [42]:
base_model_pred_valid.shape

(706, 25088)

In [43]:
# checking the min and max of the extracted features
base_model_pred.min(), base_model_pred.max()

(0.0, 721.34357)

In [44]:
#get maximum value from generated features
max_val = base_model_pred.max()

In [45]:
max_val

721.34357

In [46]:
#normalizing features generated from the VGG16 model to [0,1]
base_model_pred = base_model_pred / max_val
base_model_pred_valid = base_model_pred_valid / max_val
base_model_pred.min(), base_model_pred.max()

(0.0, 1.0)

In [47]:
#create a sequential model
model = Sequential()
# add input layer to the model that accepts input of shape 7*7*512
model.add(InputLayer((7*7*512, )))
# add fully connected layer with 1024 neurons and relu activation
model.add(Dense(units=1024, activation='relu'))
# add fully connected layer with 2 neurons and relu activation
model.add(Dense(units=2, activation='softmax'))

In [50]:
# compile the model
model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

In [51]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 dense_1 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 25693186 (98.01 MB)
Trainable params: 25693186 (98.01 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [52]:
# train model using features generated from VGG16 model
model.fit(base_model_pred, y_train, epochs=10, validation_data=(base_model_pred_valid, y_valid))

Epoch 1/10
52/52 [==============================] - 3s 18ms/step - loss: 0.2345 - accuracy: 0.9101 - val_loss: 0.1285 - val_accuracy: 0.9518
Epoch 2/10
52/52 [==============================] - 1s 11ms/step - loss: 0.0420 - accuracy: 0.9903 - val_loss: 0.1213 - val_accuracy: 0.9589
Epoch 3/10
52/52 [==============================] - 1s 11ms/step - loss: 0.0198 - accuracy: 0.9945 - val_loss: 0.1393 - val_accuracy: 0.9518
Epoch 4/10
52/52 [==============================] - 1s 11ms/step - loss: 0.0162 - accuracy: 0.9970 - val_loss: 0.1454 - val_accuracy: 0.9476
Epoch 5/10
52/52 [==============================] - 1s 11ms/step - loss: 0.0238 - accuracy: 0.9970 - val_loss: 0.1307 - val_accuracy: 0.9561
Epoch 6/10
52/52 [==============================] - 1s 11ms/step - loss: 0.0107 - accuracy: 0.9964 - val_loss: 0.1370 - val_accuracy: 0.9490
Epoch 7/10
52/52 [==============================] - 1s 11ms/step - loss: 0.0073 - accuracy: 0.9976 - val_loss: 0.1460 - val_accuracy: 0.9561
Epoch 8/10
52

## 6. Get Predictions

In [53]:
# get predictions
predictions = model.predict(base_model_pred_valid)

predictions = predictions.reshape(-1,)

# converting probabilities to classes
predictions_int = predictions >= 0.5
predictions_int = predictions_int.astype(int)
#show predictions
predictions_int

23/23 [==============================] - 0s 2ms/step


array([1, 0, 0, ..., 0, 1, 0])

## Summary of the video

1. <font color='blue'>Import neccessary libraries
2. Load the data</font>
3. <font color='#999900'>Preprocess the data</font>
4. <font color='red'>Load weights of pre-trained model
5. Fine tune the model for the current problem</font>
6. <font color='blue'>Get predictions</font>